# SVM ensembles
## HiggsML challenge

In [17]:
# interpreter
%matplotlib inline

# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn import svm

## Loading data, preprocessing and feature generation

In [8]:
# LOADING
dirName = '../../data/'
fileName = dirName + 'training.csv'
data = pd.read_csv(fileName)

# PREPROCESSING
# replace missing values with NaN
data = data.replace(-999.0, np.nan)

# learning data
X = data.copy()
del X['EventId']
del X['Weight']
del X['Label']
y = data['Label']
w = data['Weight']

# replace missing values with NaN
X = X.replace(-999.0, np.nan)

# missing_values is the value of your placeholder, strategy is if you'd like mean, median or mode, and axis=0 means it calculates the imputation based on the other feature values for that sample
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)
missingX = imp.transform(X)

# scale for feature generation
def scale():
    scaler = preprocessing.StandardScaler()
    missingX_scaled = scaler.fit_transform(missingX)
    return missingX_scaled


missingX_scaled = scale()

# FEATURE GENERATION
# (8) from the thesis

# 1 - take original missingX
# 2 - add features based on X and missingX
# 3 - perform scaling
# 4 - use original code above (make sets + SVM)

# remember original missingX
originalMissingX = missingX.copy()

# add missing value information (1 present, 0 missing)
# functions
def hasNaN(name):
    values = list(np.isnan(X[name]).values)
    for i in values:
        if (i == True):
            return True
    return False

def addMissing():
    missX = missingX.copy()
    cols = list(X.columns.values)
    for col in cols:
        if hasNaN(col):
            print('Adding additional feature - missing value:', col)
            myCol = np.isnan(X[col]).map({True: 1, False: 0})
            missX = np.c_[missX, myCol]
            additionalNames.append("Manjkajoči " + col)
    return(missX)

# FG - missing
missingX = originalMissingX.copy();
# add columns with missing number
additionalNames = [];
missX = addMissing()
# write to the table
missingX = missX.copy()
def filterMissing(relevantMissing):
    missX = missingX.copy()
    newAdditionalNames = []
    deletedN = 0
    for x in additionalNames:
        if not (x in relevantMissing):
            nIndex = additionalNames.index(x) - deletedN
            aIndex = nIndex + 30 # we are keeping original features
            print(x)
            
            # remove column
            missX = np.delete(missX, aIndex, 1)
            deletedN = deletedN + 1
        else:
            newAdditionalNames.append(x)
                
    return(missX, newAdditionalNames)

# filter only relevant features
relevantMissing = ['Manjkajoči DER_mass_MMC']
missingX, additionalNames = filterMissing(relevantMissing)


# FG - x^2
# add columns with x^2
def x2kernel(x):
    return(x*x)
def addOneX(name, kernel):
    missX = missingX.copy()
    cols = list(X.columns.values)
    i = 0
    for col in cols:
        myCol = kernel(missingX[:,i])
        print(name + ' ' + col)
        missX = np.c_[missX, myCol]
        additionalNames.append(name + " - " + col)
        i = i + 1
    return(missX)

def addOneXNormalized(name, kernel):
    missX = missingX.copy()
    cols = list(X.columns.values)
    i = 0
    for col in cols:
        myCol = kernel(missingX_scaled[:,i])
        print(name + ' ' + col)
        missX = np.c_[missX, myCol]
        additionalNames.append(name + " - " + col)
        i = i + 1
    return(missX)
missX = addOneX('$x^2$', x2kernel)
missingX = missX.copy()
# filter only relevant features
relevantMissing = relevantMissing + ['$x^2$ - DER_mass_MMC',
 '$x^2$ - DER_mass_vis',
 '$x^2$ - DER_deltar_tau_lep',
 '$x^2$ - DER_pt_tot',
 '$x^2$ - PRI_met']
missingX, additionalNames = filterMissing(relevantMissing)



# FG - e^x
# add columns with e^x
def exkernel(x):
    return(np.exp(x))
missX = addOneXNormalized('$e^x$', exkernel)
missingX = missX.copy()
# filter only relevant features
relevantMissing = relevantMissing + ['$e^x$ - DER_sum_pt', '$e^x$ - PRI_met_sumet']
missingX, additionalNames = filterMissing(relevantMissing)


# FG - sqrt(x)
# add columns with sqrt(x)
def sqrtkernel(x):
    xArray = []
    for val in x:
        if (val > 0):
            xArray.append(np.sqrt(val))
        else:
            xArray.append(-np.sqrt(-val))
    return xArray;
missX = addOneX('$\sqrt{x}$', sqrtkernel)
missingX = missX.copy()
# filter only relevant features
relevantMissing = relevantMissing + ['$\\sqrt{x}$ - DER_mass_MMC',
 '$\\sqrt{x}$ - DER_mass_vis',
 '$\\sqrt{x}$ - DER_deltar_tau_lep',
 '$\\sqrt{x}$ - DER_pt_ratio_lep_tau',
 '$\\sqrt{x}$ - PRI_met']
missingX, additionalNames = filterMissing(relevantMissing)


# FINAL SCALING
# scale the data for SVM

missingX_scaled = scale()

Adding additional feature - missing value: DER_mass_MMC
Adding additional feature - missing value: DER_deltaeta_jet_jet
Adding additional feature - missing value: DER_mass_jet_jet
Adding additional feature - missing value: DER_prodeta_jet_jet
Adding additional feature - missing value: DER_lep_eta_centrality
Adding additional feature - missing value: PRI_jet_leading_pt
Adding additional feature - missing value: PRI_jet_leading_eta
Adding additional feature - missing value: PRI_jet_leading_phi
Adding additional feature - missing value: PRI_jet_subleading_pt
Adding additional feature - missing value: PRI_jet_subleading_eta
Adding additional feature - missing value: PRI_jet_subleading_phi
Manjkajoči DER_deltaeta_jet_jet
Manjkajoči DER_mass_jet_jet
Manjkajoči DER_prodeta_jet_jet
Manjkajoči DER_lep_eta_centrality
Manjkajoči PRI_jet_leading_pt
Manjkajoči PRI_jet_leading_eta
Manjkajoči PRI_jet_leading_phi
Manjkajoči PRI_jet_subleading_pt
Manjkajoči PRI_jet_subleading_eta
Manjkajoči PRI_jet_sub

# Use - binary classification result
## Learn and save 9 SVM models
25k lines to learn EACH; 
Use RBF(8) - most successful from the tests.

In [18]:
# validation
# compute AMS
def ams(s, b):
    from math import sqrt,log
    if b==0:
        return 0

    return sqrt(2*((s+b+10)*log(1+float(s)/(b+10))-s))

# compute all measures
def validate(predicted, real, weights):
    sumsig = 0.
    sumbkg = 0.
    tp = 0.
    tn = 0.
    fp = 0.
    fn = 0.
    precision = 0.
    recall = 0.
    acc = 0.
    
    if (predicted.shape[0] != real.shape[0]):
        raise Exception
    
    for i in range(predicted.shape[0]):
        if predicted[i] == "s":
            if real[i] == "s":
                sumsig += weights[i]
                tp += 1
            else:
                sumbkg += weights[i]
                fp += 1
        else:
            if real[i] == "s":
                fn += 1
            else:
                tn += 1
    
    print(tp, fp, fn, tn)
    
    # calculate scores
    amsscore = ams(sumsig * 10, sumbkg * 10)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    acc = (tp + tn) / (tp + fp + tn + fn)
    f1score = (2 * precision * recall)/(precision + recall)

    printScores(tp, tn, fp, fn, precision, recall, acc, f1score, amsscore)
    
    return amsscore

def printScores(tp, tn, fp, fn, precision, recall, acc, f1score, amsscore):
    all = tp + tn + fp + fn
    print(tp/all * 100, tn/all * 100, fp/all * 100, fn/all * 100)
    print(precision, recall, acc, f1score)
    print(amsscore)


In [28]:
# make test dataset
Xtest = Xtest = missingX_scaled[-25000:]
ytest = ytest = y[-25000:]
wtest = w[-25000:]

# learnSVM, n = model number 1-9
def learnSVM(n):
    # number of samples per model
    numSamples = 25000
    
    # define dataset
    startOffset = (n) * numSamples + 1
    endOffset = startOffset + numSamples
    
    Xtrain = missingX_scaled[startOffset:endOffset]
    ytrain = y[startOffset:endOffset]
    wtrain = w[startOffset:endOffset]
    ytrainVals = ytrain.replace(to_replace=['s','b'],value=[1,0])
    
    # start learning
    C = 1.0
    # SVM RBF
    clf = svm.SVC(verbose=1)
    clf.fit(Xtrain, ytrainVals)
    
    return clf;

# validation
def validateModel(clf):
    predicted = clf.predict(Xtest)
    predictedV = pd.Series(predicted).map({1: 's', 0: 'b'})
    validate(predictedV, np.array(ytest), np.array(wtest))

# save model
def saveModel(name, n, clf):
    joblib.dump(clf, name + '-' + str(n) + '.pkl') 

# learin all 9 models
clfs = []
def learnAllSVMs():
    for i in range(9):
        print("Model ", i)
        clfs.append(learnSVM(i))
        validateModel(clfs[i])
        saveModel("SVM8", i, clfs[i])
        print("-----------")

learnAllSVMs()

Model  0
[LibSVM]6098.0 1834.0 2511.0 14557.0
24.392 58.228 7.335999999999999 10.044
0.7687846696923852 0.7083284934371007 0.8262 0.7373193881869293
2.7843998235755807
-----------
Model  1
[LibSVM]6086.0 1784.0 2523.0 14607.0
24.343999999999998 58.428000000000004 7.136000000000001 10.091999999999999
0.7733163913595934 0.7069346033221048 0.82772 0.7386370532192488
2.831280085527397
-----------
Model  2
[LibSVM]5998.0 1706.0 2611.0 14685.0
23.992 58.74 6.824 10.444
0.7785565939771547 0.6967127424788012 0.82732 0.7353644332740759
2.8526817064556647
-----------
Model  3
[LibSVM]6099.0 1819.0 2510.0 14572.0
24.396 58.288 7.276000000000001 10.040000000000001
0.7702702702702703 0.7084446509466837 0.82684 0.7380649845707026
2.7485440921925
-----------
Model  4
[LibSVM]6021.0 1697.0 2588.0 14694.0
24.084 58.775999999999996 6.787999999999999 10.352
0.7801243845555843 0.6993843651992101 0.8286 0.7375512954002572
2.8416114891935025
-----------
Model  5
[LibSVM]6017.0 1713.0 2592.0 14678.0
24.068 5

In [40]:
# validation with voting
def validateEnsembleVoting(clfs):
    N = len(clfs)
    for i in range(N):
        print('Validating Model', i)
        predicted = clfs[i].predict(Xtest)
        predictedV = pd.Series(predicted)
        if (i == 0):
            votes = predictedV.copy();
        else:
            votes = np.c_[votes, predictedV]
    
    # voting count
    predictedAll = pd.Series(np.mean(votes, axis=1)>0.5).map({True: 's', False: 'b'})
    validate(predictedAll, np.array(ytest), np.array(wtest))
    
    return(votes)
    
votes = validateEnsembleVoting(clfs)

Validating Model 0
Validating Model 1
Validating Model 2
Validating Model 3
Validating Model 4
Validating Model 5
Validating Model 6
Validating Model 7
Validating Model 8


In [52]:
predictedAll = pd.Series(np.mean(votes, axis=1)>0.5).map({True: 's', False: 'b'})
validate(predictedAll, np.array(ytest), np.array(wtest))

6099.0 1719.0 2510.0 14672.0
24.396 58.687999999999995 6.876 10.040000000000001
0.7801227935533385 0.7084446509466837 0.83084 0.7425579838071467
2.853868020219204


2.853868020219204

## Cascade classification model

In [53]:
def learnMetaSVM(clfs):
    # number of samples per model
    numSamples = 225000
    
    # define dataset
    startOffset = 1
    endOffset = numSamples
    
    Xtrain = missingX_scaled[startOffset:endOffset]
    ytrain = y[startOffset:endOffset]
    wtrain = w[startOffset:endOffset]
    ytrainVals = ytrain.replace(to_replace=['s','b'],value=[1,0])
    
    # create training set for meta classifier
    N = len(clfs)
    for i in range(N):
        print('Calculating Model', i)
        predicted = clfs[i].predict(Xtrain)
        predictedV = pd.Series(predicted)
        if (i == 0):
            Xvotes = predictedV.copy();
        else:
            Xvotes = np.c_[Xvotes, predictedV]
    
    # create meta classifier
    clf = svm.SVC(verbose=1)
    clf.fit(Xvotes, ytrainVals)
    
    return clf;

metaclf = learnMetaSVM(clfs)

Validating Model 0
Validating Model 1
Validating Model 2
Validating Model 3
Validating Model 4
Validating Model 5
Validating Model 6
Validating Model 7
Validating Model 8
[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=1)

In [63]:
# validation of cascade model
def validateCascadeModel(clf):
    predicted = clf.predict(votes)
    predictedV = pd.Series(predicted).map({1: 's', 0: 'b'})
    validate(predictedV, np.array(ytest), np.array(wtest))

In [60]:
# hack - DO NOT USE!!!
metaclf = Out[53]

In [64]:
validateCascadeModel(metaclf)

6109.0 1761.0 2500.0 14630.0
24.436 58.52 7.0440000000000005 10.0
0.7762388818297332 0.7096062260425137 0.82956 0.7414284847381516
2.842482295918227


(25000, 9)

In [ ]:
(25000, 9)